In [ ]:
# Google Colab을 기준으로 작성
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

# 세부적인 상품 및 구매자의 정보를 한번에 얻기 위해서 새로운 DataFrame 저장
demo = pd.read_csv('/content/drive/MyDrive/롯데2022/LPOINT_BIG_COMP_01_DEMO.csv')
pdde = pd.read_csv('/content/drive/MyDrive/롯데2022/LPOINT_BIG_COMP_02_PDDE.csv')
pd_clac = pd.read_csv('/content/drive/MyDrive/롯데2022/LPOINT_BIG_COMP_04_PD_CLAC.csv')

demo_pdde = pd.merge(pdde, demo, on='cust')
purchase = pd.merge(demo_pdde, pd_clac, on='pd_c')

In [ ]:
purchase['month'] = pd.to_datetime(purchase['de_dt'].astype(str), format='%Y/%m/%d').dt.month

import numpy as np
conditions = [
    (purchase['month'] == 1 )|(purchase['month'] == 2 )|(purchase['month'] == 12 ),
    (purchase['month'] == 3)|(purchase['month'] == 4 )|(purchase['month'] == 5 ),
    (purchase['month'] == 6 )|(purchase['month'] == 7 )|(purchase['month'] == 8 ),
    (purchase['month'] == 9 )|(purchase['month'] == 10 )|(purchase['month'] == 11 ),
    ]

# create a list of the values we want to assign for each condition
values = ['겨울', '봄', '여름', '가을']

# create a new column and use np.select to assign values to it using our lists as arguments
purchase['season'] = np.select(conditions, values)

In [ ]:
now_month = pd.Timestamp.today()
now_month = now_month.month

now_season = ''
if now_month in [1,2,12]:
    now_season = '겨울'
elif now_month in [3,4,5]:
    now_season = '봄'
elif now_month in [6,7,8]:
    now_season = '여름'
elif now_month in [9,10,11]:
    now_season = '가을'

# 제품에 대한 소/중/대분류 값을 한번에 보기 위해 'string'열 생성
purchase['string'] = purchase['pd_nm'] + " " + purchase['clac_mcls_nm'] + " " + purchase['clac_hlv_nm'] + ' | '

In [ ]:
#추천하고자 하는 고객의 고유 식별 번호 입력 ('cust')
while True:
    cust_input = input("고객번호: ")
    channel_selector = input("Online/Offline: ")
    
    if (channel_selector == 'Offline') | (channel_selector == 'offline'):
        channel_selector = 1
    else:
        channel_selector = 2
    
    check = purchase.loc[(purchase['cust'] == cust_input) & (purchase['chnl_dv'] == channel_selector)]
    if len(check.index) > 0:
        break
    else:
        print('No previous data for corresponding customer and channel')
        print('Please re-enter a valid condition')    


age = list(demo[demo.cust == cust_input].ages)
sex = list(demo[demo.cust == cust_input].ma_fem_dv)


In [ ]:
def channel_filter(purchase):
    
    # 입력된 고객(cust_input) 동일한 조건을 가진 고객만 필터
    # 나이 필터링
    age_filter = purchase[purchase.ages == age[0]] 

    # 나이 + 성별 필터링
    age_sex_filter = age_filter[age_filter.ma_fem_dv == sex[0]] 
    
    # 나이 + 성별 + 현재 계절 필터링
    final_filter = age_sex_filter[age_sex_filter.season == now_season]
    
    return final_filter

In [ ]:
# 구매 채널에 맞는 데이터 필터링
if channel_selector == 2:
    purchase_online = purchase[purchase.chnl_dv == 2].sort_values('pd_nm')
    
    final_filter = channel_filter(purchase_online)
    
elif channel_selector == 1:
    purchase_offline = purchase[purchase.chnl_dv == 1].sort_values('pd_nm')

    final_filter = channel_filter(purchase_offline)


In [ ]:
# 필터에 해당되는 고객들의 구매 품목 목록을 생성

cust_list = []
data = {}
for i in list(final_filter.cust):
    if i not in cust_list:
        str_to_append = final_filter[final_filter.cust == i]['string'].sum()
        data[i] = str_to_append
        cust_list.append(i)
    else:
        continue

purchase_filter = pd.DataFrame(data, index=[0])
purchase_filter = purchase_filter.T.reset_index()
purchase_filter = purchase_filter.rename(columns={'index':'cust',0:'string_combined'})

In [ ]:
# 추천 고객이 현재 계절과 일치하는 구매기록이 없을때
if check[check['season'] == now_season].empty:
    cust_recommend = purchase[(purchase['cust'] == cust_input) & (purchase['chnl_dv'] == channel_selector)]['string'].sum()
    purchase_filter.loc[len(purchase_filter.index)] = [cust_input, cust_recommend]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 자연어 처리 위해 vectorizer 사용
# ppt에 왜 countervectorizer 말고 tfidfvectorizer를 썼는지 서술이 필요할듯
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(purchase_filter['string_combined'])

# tfidf_matrix.shape # 개수를 보여줄거야?? ppt에서는 예시를 보여줄때는 필요한 부분같음

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(purchase_filter.index, index=purchase_filter['cust']).drop_duplicates()

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):

    # Get the index of the customer that matches the title
    idx = indices[title]
    
    # Get the pairwise similarity scores of all customers' order with that specific customer's order
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # We set minimum sample size to obtain a meaningfull result as 30
    return sim_scores[:30]

In [ ]:
from collections import Counter

# Getting the index of input customer
idx = purchase_filter[purchase_filter.cust == cust_input].index

# Using numpy to get unique items that are previously bought by the input customer
lst_contain = purchase_filter.loc[idx[0], 'string_combined'].split('|')
lst_contain = np.array(lst_contain)
lst_contain = list(np.unique(lst_contain))
lst_contain = [i.strip() for i in lst_contain]

# Extracting customers that matches the conditions of the input customer
similar_user_list = list(get_recommendations(cust_input))

count = {}
similar_user_list
for i in similar_user_list:
    if i == cust_input:
        continue
    else:
        lst_compare = purchase_filter.loc[i[0], 'string_combined'].split('|')
        lst_compare = [k.strip() for k in lst_compare]
        for j in lst_compare:
            if j not in lst_contain:
                # using weighted score to calculate how frequent each item each choosen by similar customers
                try: count[j] += 1*i[1]
                except: count[j] = 1*i[1]

# pick top 5 scored items to recommend
c = Counter(count)
c = c.most_common(5)

# 결과 출력
print("--- 추천 리스트 ---")
for i,j in enumerate(c):
    print(str(i+1)+"번:"+j[0])